In [1]:
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.grid_search import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint as sp_randint
import datasets
from time import time

In [2]:
data=datasets.load_cifar_hog()
scaler = StandardScaler()
X_train = scaler.fit_transform(data.X_train)
Y_train = data.y_train
X_test = scaler.transform(data.X_test)
Y_test = data.y_test
n_feats=X_train.shape[1]

In [3]:
n_iter=20
parameters = {
    "base_estimator__max_depth": [3, None],
    "base_estimator__max_features": sp_randint(1, n_feats),
    "base_estimator__bootstrap": [True, False],
    "base_estimator__criterion": ["gini", "entropy"],
    "n_estimators":sp_randint(1,100),
    "learning_rate":np.logspace(-2,2,5),
}

In [1]:
rndf=RandomForestClassifier(n_estimators=15,n_jobs=-1,random_state=1,warm_start=True)
adaboost=AdaBoostClassifier(base_estimator=rndf,random_state=1)

NameError: name 'RandomForestClassifier' is not defined

In [6]:
random_search = RandomizedSearchCV(adaboost, param_distributions=parameters,n_iter=n_iter,n_jobs=-1)

In [ ]:
start = time()
random_search.fit(X_train, Y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter))

In [ ]:
def report(grid_scores, n_top=5):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

print report(random_search.grid_scores_)

In [ ]:
print 'Test set accuracy:'
print random_search.score(X_test,Y_test)